# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [7]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sqrt_value_cum = list()
for sqrt_value in generator:
    print(sqrt_value)
    sqrt_value_cum.append(sqrt_value)

import numpy as np
print('')
print(np.sum(sqrt_value_cum))

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979

8.382332347441762


In [8]:
# Example usage:
limit = 13
generator = square_root_generator(limit)

sqrt_value_cum = list()
for sqrt_value in generator:
    print(sqrt_value)
    sqrt_value_cum.append(sqrt_value)



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [9]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [10]:
people_1_data = list(people_1())


sum_ages_1 = sum(person['Age'] for person in people_1_data)

people_2_data = list(people_2())

combined_people_data = people_1_data + people_2_data

sum_ages_combined = sum(person['Age'] for person in combined_people_data)

sum_ages_combined

353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [11]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [44]:
import dlt

In [45]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='age_persons')

In [46]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

In [47]:
info = generators_pipeline.run(people_1(),
          table_name="persons",
          write_disposition="replace")

print(info)


2024-02-15 20:29:05,678|[WARNING              ]|133|136827823783936|dlt|reference.py|_verify_schema:357|A column record_hash in table persons in schema dlt_colab_kernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset age_persons
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708028945.301559 is LOADED and contains no failed jobs


In [48]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ persons             │
└─────────────────────┘

In [49]:
persons = conn.sql("SELECT * FROM persons").df()
display(persons)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708028945.301559,Usc3PhaJeM6KEQ,None
1,2,Person_2,27,City_A,1708028945.301559,h4y3rRicoNheeg,None
2,3,Person_3,28,City_A,1708028945.301559,beI5ZuCggYyGOA,None
3,4,Person_4,29,City_A,1708028945.301559,xjTrbgyyOD6ViQ,None
4,5,Person_5,30,City_A,1708028945.301559,8SjrBZSCil3+Zg,None


In [50]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


In [51]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='age_persons')

info = pipeline.run(people_2(),
          table_name="persons",
          write_disposition="merge",
                    primary_key='id')

2024-02-15 20:29:06,565|[WARNING              ]|133|136827823783936|dlt|reference.py|_verify_schema:357|A column record_hash in table persons in schema dlt_colab_kernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-15 20:29:06,626|[WARNING              ]|133|136827823783936|dlt|reference.py|_verify_schema:357|A column record_hash in table persons in schema dlt_colab_kernel_launcher is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


In [52]:
persons = conn.sql("SELECT * FROM persons").df()
display(persons)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708028945.301559,Usc3PhaJeM6KEQ,None
1,2,Person_2,27,City_A,1708028945.301559,h4y3rRicoNheeg,None
2,8,Person_8,38,City_B,1708028946.0743392,wpt9tp8pvODIyQ,Job_8
3,4,Person_4,34,City_B,1708028946.0743392,1VwEQiAhxN13ig,Job_4
4,5,Person_5,35,City_B,1708028946.0743392,wBFV6CAo1mVgNA,Job_5
5,7,Person_7,37,City_B,1708028946.0743392,rgmWXOR4tjWglw,Job_7
6,3,Person_3,33,City_B,1708028946.0743392,1l1AtdZ5qEb8/Q,Job_3
7,6,Person_6,36,City_B,1708028946.0743392,5f9Pjw7KSh3hwA,Job_6


In [53]:
persons['age'].sum()

266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX